In [13]:
import pandas as pd
import utils
import plotly.graph_objects as go

In [14]:
pair = "EUR_USD"
granularity = "H1"
ma_list = [8,16,32,64,128,256]

In [15]:
df = pd.read_pickle(utils.get_his_data_filename(pair, granularity))

In [16]:
non_cols = ['time', 'volume']
mod_cols = [x for x in df.columns if x not in non_cols]
df[mod_cols] = df[mod_cols].apply(pd.to_numeric)

In [17]:
df_ma = df[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c']].copy()

In [18]:
df_ma.head(5)

,time,mid_o,mid_h,mid_l,mid_c
0,2022-06-06T13:00:00.000000000Z,1.07228,1.07234,1.07054,1.07142
1,2022-06-06T14:00:00.000000000Z,1.07142,1.07239,1.06918,1.07178
2,2022-06-06T15:00:00.000000000Z,1.07178,1.07214,1.06839,1.06976
3,2022-06-06T16:00:00.000000000Z,1.06975,1.07023,1.06898,1.07006
4,2022-06-06T17:00:00.000000000Z,1.07008,1.07032,1.06945,1.06986


In [19]:
for ma in ma_list:
    df_ma[f'MA_{ma}'] = df_ma.mid_c.rolling(window=ma).mean()

In [20]:
df_ma.dropna(inplace=True)

In [21]:
df_plot = df_ma.iloc[-200:].copy()

In [22]:
df_plot.shape

(200, 11)

In [24]:
fig = go.Figure()
fig.add_trace(go.Candlestick(
    x=df_plot.time, open=df_plot.mid_o, high=df_plot.mid_h, low=df_plot.mid_l, close=df_plot.mid_c,
    line=dict(width=1), opacity=1,
    increasing_fillcolor='#24A06B',
    decreasing_fillcolor="#CC2E3C",
    increasing_line_color='#2EC886',
    decreasing_line_color='#FF3A4C'
))
for ma in ma_list:
    col = f"MA_{ma}"
    fig.add_trace(go.Scatter(
        x=df_plot.time,
        y=df_plot[col],
        line=dict(width=2),
        line_shape='spline',
        name=col
        ))
fig.update_layout(width=1000,height=400,
                  margin=dict(l=10,b=10,t=10),
                  font=dict(size=10,color="#e1e1e1"),
                  paper_bgcolor="#1e1e1e",
                  plot_bgcolor="#1e1e1e")
fig.update_xaxes(
    gridcolor="#1f292f",
    showgrid=True,fixedrange=True,rangeslider=dict(visible=False)
)
fig.update_yaxes(
    gridcolor="#1f292f",
    showgrid=True,
)
fig.show()